# Build Tech Tree and Calculate Depths

Grace Deng, August 2020

Last modified by Natalia Velez, September 2020

This file does the following:
   1. Parse the transition files (expand on categories and keep only the forward transitions)
   2. Calculates the depth (defined as the number of unique ingredients) of each object in the tech tree
   3. Calculates the empowerment of each object (defined as the number of immediate products that can be made from this object)


In [1]:
import os, re, glob, json
from os.path import join as opj
import numpy as np
import pandas as pd
import json
import networkx as nx
from tqdm import notebook

import ohol_transitions as trans
import ohol_categories as cat
import ohol_objects as obj

## Find object and transition files:

Transitions:

In [2]:
gsearch = lambda *args: glob.glob(opj(*args))
data_dir = '../../OneLifeData7/'
trans_dir  = opj(data_dir, 'transitions')
trans_files = gsearch(trans_dir, '*.txt')

print(*trans_files[:5], sep='\n')

../../OneLifeData7/transitions/67_2970.txt
../../OneLifeData7/transitions/235_1890.txt
../../OneLifeData7/transitions/441_445.txt
../../OneLifeData7/transitions/0_2671.txt
../../OneLifeData7/transitions/239_335.txt


Objects:

In [3]:
obj_dir = opj(data_dir, 'objects')
obj_files = gsearch(obj_dir, '*txt')

print(*obj_files[:5], sep='\n')

../../OneLifeData7/objects/2268.txt
../../OneLifeData7/objects/4179.txt
../../OneLifeData7/objects/2662.txt
../../OneLifeData7/objects/3556.txt
../../OneLifeData7/objects/2475.txt


## Helper Functions:

Find object ingredients, based on transition dictionary

In [4]:
def find_ingredients(value):
    ingredient_list = []
    for idx in d.items():
        for item in d[idx[0]].items():
            if item[0] != 'category':
                if value in item[1]:
                    ingredient_list.append(item[0])
    return list(set(ingredient_list))       


Returns the category for a given object

In [5]:
##helper function that returns the category for a given object
def find_parent(mydict, child):
    for i in mydict.keys():
        if child in mydict[i]:
            return i

Split dataframe:

In [6]:
def split_dataframe_rows(df,column_selectors):
    # we need to keep track of the ordering of the columns
    def _split_list_to_rows(row,row_accumulator,column_selector):
        split_rows = {}
        max_split = 0
        for column_selector in column_selectors:
            split_row = row[column_selector]
            split_rows[column_selector] = split_row
            if len(split_row) > max_split:
                max_split = len(split_row)
        for i in range(max_split):
            new_row = row.to_dict()
            for column_selector in column_selectors:
                try:
                    new_row[column_selector] = split_rows[column_selector].pop(0)
                except IndexError:
                    new_row[column_selector] = ''
            row_accumulator.append(new_row)

    new_rows = []
    df.apply(_split_list_to_rows,axis=1,args = (new_rows,column_selectors))
    new_df = pd.DataFrame(new_rows, columns=df.columns)
    return new_df

Finds the key associated with a certain value in the dictionary

In [7]:
def find_value(dictionary, value):
    mylist = []
    for item in dictionary.items():
        if value == item[1]:
            mylist.append(item[0])
    return mylist

## Parse Categories and Objects

Build a dictionary of {object: mapChance} pairs, and identify category names

In [8]:
str_extract = lambda pattern, s: re.search(pattern, s).group(0)
int_extract = lambda pattern, s: int(str_extract(pattern, s))
cat_names = {}
obj_dict = {}
all_obj = {}
for o in notebook.tqdm(obj_files):
    is_obj = re.search('nextObjectNumber|groundHeat', o) is None
    
    if is_obj:
        o_num = int_extract('[0-9]+(?=.txt)', o)
        o_data = obj.read_obj(o_num)
        obj_dict[o_num] = o_data['mapChance']
        if '@' in o_data['name'] :
            cat_names[o_num] = o_data['name']
        else:
            all_obj[o_num] = o_data['name']


Build a dictionary of {category: children} pairs

In [9]:
l = os.listdir('../../OneLifeData7/categories')
cat_arr = [int_extract('[0-9]+(?=.txt)', o) for o in l]
len(cat_arr)

258

In [10]:
perhaps_list = list()
for i in all_obj.values():
    if 'Perhaps' in i:
        perhaps_list.append(find_parent(all_obj, i))
perhaps_list

[3233, 2328, 3221, 2811, 2095]

In [11]:
#update: if parent is not a @category name, add parent name to child list
cat_dict = {}
for i in cat_arr:
    children = cat.get_children(i)
    if (i in cat_names):
        cat_dict[i] = children  
    else:
        cat_dict[i] = list(np.append(i, children))
          
len(cat_dict)

258

Helper function: Does this object ID actually correspond to a category?

In [12]:
def is_cat(name):
    if name in cat_dict:
        return True
    else:
        return False

Find natural primittives

In [13]:
nat_obj = []
for item in obj_dict.items():
    if item[1] != 0 :
        nat_obj.append(item[0])

In [14]:
for o in nat_obj:
    print('%i: %s' % (o, obj.obj_name(o)))

805: Wild Onion
2515: Calamine
2800: Wild Tomato Plant
1890: Lapis Lazuli
804: Burdock
121: Tule Reeds
3887: Jungle Expert Way Stone - +biomeSet3 gridPlacement20 &20 +expertFind
32: Big Hard Rock
1020: Snow Bank
713: Indigo
2466: Dark Nosaj - +normalOnly
3888: Arctic Expert Way Stone - +biomeSet3 gridPlacement20 &20 &p3 &p0 +expertFind
418: Wolf
136: Sapling
33: Stone
65: Lombardy Poplar Tree -Branch
2156: Mosquito Swarm
2141: Oil Palm
707: Antarctic Fur Seal
1157: Wild Bean Plant
2140: Hot Spring
50: Milkweed
1107: Teosinte
527: Willow Tree
2504: Malachite
133: Flint
942: Muddy Iron Vein - gridPlacement40 &40
1140: Wild Potato
1261: Canada Goose Pond with Egg
63: Maple Tree -Branch
2135: Rubber Tree
680: Gold Vein
2804: Wild Pepper Plant
714: Rose Madder
4272: Rubber Tree with Pepper Vine
4251: Wild Garlic
2540: Niter
2174: Turkey
530: Bald Cypress Tree
674: Limestone
4239: Seeding Wild Dill
729: Alum
791: Monolith
3886: Desert Expert Way Stone - +biomeSet3 gridPlacement20 &20 +expert

## Parse transitions

In [15]:
trans_keys = ['origActor', 'newActor', 'origTarget', 'newTarget']
trans_list = []
for f in notebook.tqdm(trans_files):
    trans_dict = trans.read_transition(f)
    trans_objs = [trans_dict[k] for k in trans_keys]
    
    # Are any of these categories?
    trans_cats = [is_cat(o) for o in trans_objs]
    #trans_cat_str = ''.join(re.findall('[A-Z]', str(trans_cats)))
    #trans_dict['isCat'] = trans_cat_str
    trans_dict['isCat'] = trans_cats
    
    trans_list.append(trans_dict)

Assemble into dataframe:

In [16]:
trans_df = pd.DataFrame(trans_list)
trans_df['isDecay'] = trans_df['autoDecaySeconds'] > 0
trans_df = trans_df[['origActor', 'origTarget', 'newActor', 'newTarget', 'isDecay', 'isTool', 'isCat']]
trans_df['catStr'] = trans_df['isCat'].apply(lambda b: ''.join(re.findall('[A-Z]', str(b))))

trans_df.head()

,origActor,origTarget,newActor,newTarget,isDecay,isTool,isCat,catStr
0,67,2970,0,2966,False,True,"[False, False, False, False]",FFFF
1,235,1890,1892,0,False,True,"[False, False, False, False]",FFFF
2,441,445,441,444,False,True,"[False, False, False, False]",FFFF
3,0,2671,2539,2665,False,False,"[False, False, False, False]",FFFF
4,239,335,327,291,False,False,"[False, False, False, False]",FFFF


Count each type of category transition:

In [17]:
cat_counts = trans_df.groupby('catStr')['isTool'].agg('count').reset_index()
cat_counts = cat_counts.rename(columns = {'isTool': 'n'})
cat_counts

,catStr,n
0,FFFF,4128
1,FFFT,22
2,FFTF,92
3,FFTT,158
4,FTFF,7
5,FTFT,1
6,FTTF,25
7,FTTT,6
8,TFFF,55
9,TFFT,38


Check examples of category transitions:

In [18]:
cat_trans = trans_df.groupby('catStr').first().reset_index()
cat_trans = pd.merge(cat_counts, cat_trans, on = 'catStr')
cat_trans = cat_trans.sort_values('n', ascending = False).reset_index(drop=True)
cat_trans

,catStr,n,origActor,origTarget,newActor,newTarget,isDecay,isTool,isCat
0,FFFF,4128,67,2970,0,2966,False,True,"[False, False, False, False]"
1,TTFF,190,394,3321,394,3333,False,True,"[True, True, False, False]"
2,FFTT,158,2820,519,2821,519,False,False,"[False, False, True, True]"
3,FFTF,92,-1,1769,0,0,True,False,"[False, False, True, False]"
4,TFFF,55,405,254,0,262,False,True,"[True, False, False, False]"
5,TFFT,38,1904,1884,659,1908,False,True,"[True, False, False, True]"
6,FTTF,25,0,780,778,0,False,False,"[False, True, True, False]"
7,FFFT,22,0,1939,0,1948,False,False,"[False, False, False, True]"
8,TTTT,13,3775,3774,3773,3777,False,False,"[True, True, True, True]"
9,TFTT,12,4558,4548,0,4546,False,True,"[True, False, True, True]"


## Build Tech Tree and calculate Depth

### Expand categories

If an object in a transition refers to a category, replace it with a list of objects within the category

In [19]:
tech_tree_df = trans_df.copy()
tech_tree_df = tech_tree_df[trans_keys + ['isCat']]
tech_tree_df = tech_tree_df.astype('object')

for i,row in notebook.tqdm(tech_tree_df.iterrows(), total=tech_tree_df.shape[0]):
    
    for j,is_cat in zip(trans_keys, row['isCat']):
        if is_cat:
            cat_id = row[j]
            tech_tree_df.at[i,j] = cat_dict[cat_id]

print(tech_tree_df.shape)
tech_tree_df.head()


(4769, 5)


,origActor,newActor,origTarget,newTarget,isCat
0,67,0,2970,2966,"[False, False, False, False]"
1,235,1892,1890,0,"[False, False, False, False]"
2,441,441,445,444,"[False, False, False, False]"
3,0,2539,2671,2665,"[False, False, False, False]"
4,239,327,335,291,"[False, False, False, False]"


Remove natural decay transitions:

In [20]:
tech_tree_df = tech_tree_df.query('(newActor != 0) or (newTarget != 0)')

Remove tools from transitions:

In [21]:
tech_tree_df.loc[tech_tree_df.newTarget == tech_tree_df.origActor, 'newTarget'] = ''
tech_tree_df.loc[tech_tree_df.newTarget == tech_tree_df.origTarget, 'newTarget'] = ''
tech_tree_df.loc[tech_tree_df.newActor == tech_tree_df.origTarget, 'newActor'] = ''
tech_tree_df.loc[tech_tree_df.newActor == tech_tree_df.origActor, 'newActor'] = ''

In [22]:
print(tech_tree_df.shape)
tech_tree_df.head(n = 20)

(4646, 5)


,origActor,newActor,origTarget,newTarget,isCat
0,67,0,2970,2966,"[False, False, False, False]"
1,235,1892,1890,0,"[False, False, False, False]"
2,441,,445,444,"[False, False, False, False]"
3,0,2539,2671,2665,"[False, False, False, False]"
4,239,327,335,291,"[False, False, False, False]"
5,"[402, 40]",0,254,262,"[True, False, False, False]"
6,0,236,4302,4303,"[False, False, False, False]"
7,0,2356,2359,2243,"[False, False, False, False]"
8,-1,0,420,429,"[False, False, False, False]"
9,4542,4543,-1,4535,"[False, False, False, False]"


expand on perhaps items (treated as category names)

In [24]:
trans_df.query('(newTarget in @perhaps_list) or (newActor in @perhaps_list)')

,origActor,origTarget,newActor,newTarget,isDecay,isTool,isCat,catStr
1529,-1,1195,0,3221,True,False,"[False, False, False, True]",FFFT
2375,-1,2810,0,2811,True,False,"[False, False, False, True]",FFFT
2546,-1,3232,0,3233,True,False,"[False, False, False, True]",FFFT
2719,-1,2331,0,2328,True,False,"[False, False, False, True]",FFFT
3606,-1,2101,0,2095,True,False,"[False, False, False, True]",FFFT
4142,-1,2303,0,2328,True,False,"[False, False, False, True]",FFFT


parse transitions data frame (expand on categories)

In [ ]:
## TODO: This is my attempt to clean up this code - NV
# edge_cases = []
# expanded_tech_list = []
# new_cols = ['newTarget', 'newActor']

# for idx,row in tech_tree_df.iterrows():
    
#     # Which of these columns contain lists? (i.e., need to be expanded)
#     columns_to_expand = []
#     for col in trans_keys:
#         if type(row[col]) is list:
#             columns_to_expand.append(col)
            
#     # CASE 1: All individual components - proceed!
#     if not len(columns_to_expand):
#         expanded_tech_list.append(row)
#     # CASE 2: Exactly 1 components is a category
#     elif (len(columns_to_expand) == 1) and (columns_to_expand[0] in new_cols):
#         target = columns_to_expand[0]
#         items = row[target].copy()
#         print(row)
        
#         # Special case: If the product is "perhaps __", remove it
#         if items[0] in perhaps_list:
#             print(target)
#             items.pop(0)
        
            
        
# #     #category id only in product: category id is actually item id
# #     elif (len(target_columns) == 1) and ((target_columns=='newTarget') or (target_columns=='newActor')):
# #         if (target_columns=='newTarget'): #if the product is in target
# #             if curr_line['newTarget'][0] in perhaps_list: #special cases when the product is 'Perhaps xxx'
# #                 curr_line['newTarget'] = curr_line['newTarget'][1:]
# #                 for k in curr_line.values: #first turn every cell into a list obj
# #                     if type(k) is int:
# #                         k = [k]
# #                 sub = split_dataframe_rows(curr_line.to_frame().T, target_columns) #split row
# #                 newdf = newdf.append(sub)#append to dataframe
# #             else: #normal cases
# #                 curr_line['newTarget'] = curr_line['newTarget'][0] #only keep the first element of the list
# #                 newdf = newdf.append(curr_line.to_frame().T) #append to dataframe
# #         elif (target_columns=='newActor'): #if the product is in actor
# #             curr_line['newActor'] = curr_line['newActor'][0]
# #             newdf = newdf.append(curr_line.to_frame().T) #append to dataframe
        
# #     #more categories in equation and they match in lengths
# #     else:
# #         length = len(curr_line[target_columns[0]])
# #         if all(len(lst) == length for lst in curr_line[target_columns]): #check that they are of the same lengths
# #             for k in curr_line.values: #first turn every cell into a list obj
# #                 if type(k) is int:
# #                     k = [k]
# #             sub = split_dataframe_rows(curr_line.to_frame().T, target_columns) #split row
# #             newdf = newdf.append(sub)#append to dataframe
# #         else:
# #             edge_cases = np.append(edge_cases, i) #edge cases are not included in newdf
        


In [34]:
mydf = tech_tree_df.copy()
edge_cases = []
newdf = pd.DataFrame(columns = ['origActor','origTarget','newActor','newTarget'])
for i in range(0,len(mydf)):
    curr_line = mydf.iloc[i]
    target_columns = []
    for j in ['origActor','origTarget','newActor','newTarget']:
        if (type(curr_line[j]) is list):
            if len(curr_line[j]):
                target_columns = np.append(target_columns,j)
            
    #nothing in the equation has a category id       
    if (len(target_columns) == 0):
        newdf = newdf.append(curr_line.to_frame().T) #append to dataframe
        
    #category id only in product: category id is actually item id
    elif (len(target_columns) == 1) and ((target_columns=='newTarget') or (target_columns=='newActor')):
        if (target_columns=='newTarget'): #if the product is in target
            if curr_line['newTarget'][0] in perhaps_list: #special cases when the product is 'Perhaps xxx'
                curr_line['newTarget'] = curr_line['newTarget'][1:]
                for k in curr_line.values: #first turn every cell into a list obj
                    if type(k) is int:
                        k = [k]
                sub = split_dataframe_rows(curr_line.to_frame().T, target_columns) #split row
                newdf = newdf.append(sub)#append to dataframe
            else: #normal cases
                curr_line['newTarget'] = curr_line['newTarget'][0] #only keep the first element of the list
                newdf = newdf.append(curr_line.to_frame().T) #append to dataframe
        elif (target_columns=='newActor'): #if the product is in actor
            curr_line['newActor'] = curr_line['newActor'][0]
            newdf = newdf.append(curr_line.to_frame().T) #append to dataframe
        
    #more categories in equation and they match in lengths
    else:
        length = len(curr_line[target_columns[0]])
        if all(len(lst) == length for lst in curr_line[target_columns]): #check that they are of the same lengths
            for k in curr_line.values: #first turn every cell into a list obj
                if type(k) is int:
                    k = [k]
            sub = split_dataframe_rows(curr_line.to_frame().T, target_columns) #split row
            newdf = newdf.append(sub)#append to dataframe
        else:
            edge_cases = np.append(edge_cases, i) #edge cases are not included in newdf

remove situations where nothing is produced

In [42]:
for _,row in newdf.iterrows():
    for key in trans_keys:
        if type(key) is list:
            print(row)

In [43]:
newdf = newdf[trans_keys]
newdf = newdf.astype('int32')
newdf.query('(newActor != "") or (newTarget != "")')

ValueError: setting an array element with a sequence.

parse cases where the lengths of both sides do not match

In [ ]:
edge = tech_tree_newdf.iloc[edge_cases].copy()
edge = edge.query('newTarget != 1947') #ignore the cards for now
edge

In [ ]:
cols = ['origActor','origTarget','newActor','newTarget']
edge_df = pd.DataFrame(columns = cols)

for j in range(0, len(edge)):
    lengths = [1,1,1,1]
    curr_line = edge.iloc[j]
    for i in range(0,len(curr_line.values)):
        if type(curr_line[i]) is list:
            lengths[i] = len(curr_line[i])

    num_repeat = np.product(np.unique(lengths))
    mat = -np.ones([num_repeat, 4]).astype('object')
    for i in range(0,4):
        if lengths[i] != max(lengths):
            mat[:,i] = np.repeat(curr_line[i], num_repeat/lengths[i])
        else:
            mat[:,i] = curr_line[i]*int(num_repeat/lengths[i])
    sub = pd.DataFrame(mat, columns = cols)
    edge_df = edge_df.append(sub)

In [ ]:
edge_df.head(n= 10)

In [ ]:
newdf = newdf.append(edge_df)
len(newdf)

save to csv

In [ ]:
newdf.to_csv('tech_outputs/transition.csv')

## Calculate Depth

make a list of items

In [ ]:
max_item_id = max(list(all_obj.keys()))
print(max_item_id)

In [ ]:
#replace negative values
newdf = newdf.replace(-1, max_item_id+1)
newdf = newdf.replace(-2, max_item_id+2)

In [ ]:
#make a list of all objects
items = np.arange(max_item_id+3)
print(max(items))

### create a nested dictionary of ingredients - products pair

In [ ]:
#dictionary problem has been fixed. Ingredients will not override
#Update: also expands on situations where a category is in one side of the transition
d = {}
for item in all_obj:
    d[item] = {}
    recipy = newdf.query('(origActor == @item) or (origTarget == @item)')

    if len(recipy):
        for j in range(0,len(recipy)):
            ingredients = recipy.iloc[j].tolist()[0:2]
            ingredients.remove(item)
            ingredient = ingredients[0]
            products = recipy.iloc[j].tolist()[2:4]
            if "" in products:
                products.remove("")
            if ingredient in d[item].keys(): #prevent override
                d[item][ingredient] = list(map(int, set(np.append(d[item][ingredient], products))))
            else:
                d[item][ingredient] = products         
                

### loop over all pairs

In [ ]:
#initialize depth
orig_depth = np.inf
depth = {}
transition_from = {} #this calculates the shortest path
all_products = {} #this calculates all paths
for item in items:
    if item in nat_obj:
        depth[item] = 0
        transition_from[item] = [item]
    else:
        depth[item] = orig_depth
        transition_from[item] = []
    all_products[item] = []
depth[0] = 0 #empty hand
depth[len(depth)-1] = 0 #empty ground
depth[len(depth)-2] = 0 #empty (for animals)

In [ ]:
root_items = nat_obj
order_list = []
adj_dict = {}
while len(root_items):
    #take an item from root and remove it
    item = root_items[0]
    root_items = np.delete(root_items,0)

    #put all products associated with this item into root
    for key in d[item]: #looping through other ingredients
        #calculate the depths of the products
        for product in d[item][key]:

            #if this product is made of known-depth ingredients, add this product to root_items list
            if depth[product] > depth[item] + depth[key] + 1:
                depth[product] = depth[item] + depth[key] + 1  
                
                all_products[key] = np.append(all_products[key],product)
                all_products[item] = np.append(all_products[item],product)
                transition_from[product] = product
                transition_from[product] = np.append(transition_from[product], list(set(transition_from[key])))
                transition_from[product] = np.append(transition_from[product], list(set(transition_from[item])))
#                 print(transition_from[product])
                adj_dict[product] = [item, key]

                root_items = np.append(root_items, product)
                order_list = np.append(order_list,product)
                

check all the uncraftable items

In [ ]:
not_updated = find_value(depth, np.inf)
len(not_updated)

turn values to list, turn keys to string, and remove the product itself from its ingredients

In [ ]:
for key in transition_from.keys():
    transition_from[key] = [int(i) for i in transition_from[key] if (i != key)]
transition_from = {str(k):list(set(v)) for k,v in transition_from.items()}

calculate the number of immediate products that can be made from this object

In [ ]:
emp = pd.DataFrame(columns = ['id','name','num_imme_products'])
for i in all_obj:
    emp = emp.append({'id':i, 'name':all_obj[i], 'num_imme_products':len(all_products[i])}, ignore_index = True)

In [ ]:
emp = emp.sort_values(by = 'num_imme_products', ascending = False).reset_index(drop = True)
emp.head()

In [ ]:
all_products = {str(k):list(v) for k,v in all_products.items()}

## Save to csv

In [ ]:
#for later plotting tech tree use
with open('tech_outputs/adj.csv', 'w') as f:
    f.write("ingredient1,ingredient2,product\n")
    for key in adj_dict.keys():
        f.write("%s,%s,%s\n"%(adj_dict[key][0], adj_dict[key][1], key))

In [ ]:
#save this in case it's useful. Here depth(obj) = depth(origActor) +depth(origTarget) +1
with open('tech_outputs/depth.csv', 'w') as f:
    f.write("Id,Depth,Name\n")
    for key in depth.keys():
        if key in all_obj:
            f.write("%s,%s,%s\n"%(key, depth[key], all_obj[key]))

In [ ]:
with open('tech_outputs/num_unique_ingredients.csv', 'w') as f:
    f.write("id,name,num_ingredients\n")
    for key in transition_from.keys():
        if key in all_obj:
            f.write("%s,%s,%s\n"%(key, all_obj[key], len(transition_from[key])))

In [ ]:
with open('tech_outputs/ingredients.txt', 'w') as outfile:
    json.dump(transition_from, outfile)

In [ ]:
with open('tech_outputs/products.txt', 'w') as outfile:
    json.dump(all_products, outfile)

In [ ]:
emp.to_csv('tech_outputs/empowerment.csv')

## Check:

In [ ]:
# with open('tech_outputs/ingredients.p', 'rb') as fp:
#     data = pickle.load(fp)

In [ ]:
emp_df = pd.read_csv('tech_outputs/empowerment.csv', index_col = 0)
emp_df = emp_df.sort_values(by = 'num_imme_products', ascending = False).reset_index(drop = True)
emp_df.head(n = 10)

In [ ]:
trans_df = pd.read_csv('tech_outputs/num_unique_ingredients.csv')
sorted_df = trans_df.sort_values(by = 'num_ingredients', ascending = False).reset_index(drop = True)
sorted_df.head()

In [ ]:
for i in not_updated:
    try:
        print(obj.read_obj(str(i))['name'])
    except:
        SyntaxError

In [ ]:
hardest_item = max([i for i in list(depth.values()) if i < np.inf])
myitem = find_value(depth, hardest_item, 0)[0]
tech_df.query('(newTarget == @myitem) or (newActor == @myitem)')